In [1]:
# ! pip install git+https://github.com/deepset-ai/haystack.git
#! pip install farm-haystack

/bin/bash: pip: command not found


In [2]:
from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

/home/austin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/austin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/austin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/austin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [40]:
# Recommended: Start Elasticsearch using Docker
# ! docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.6.2

13479296ac0cc1ddca4d3005ac6cb554683a7038e02692e099b4381e2df06651


In [41]:
# Connect to Elasticsearch

from haystack.database.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

07/23/2020 14:31:43 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.492s]


In [43]:
import json
with open('data.json') as f:
    data = json.load(f)
    

In [44]:
for article in data:
    filename = article['url'].split('/')[-1]
    with open('articles/%s.txt' % filename, 'w') as f:
        print(article['text'], file=f)

In [46]:
# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
doc_dir = 'articles'
dicts = convert_files_to_dicts(dir_path=doc_dir, split_paragraphs=True)
# document_store.write_documents(dicts)

07/23/2020 14:32:05 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.470s]


In [47]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [48]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

07/23/2020 14:32:22 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
07/23/2020 14:32:22 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
07/23/2020 14:32:27 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
07/23/2020 14:32:32 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
07/23/2020 14:32:33 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
07/23/2020 14:32:33 - INFO - farm.infer -   Got ya 3 parallel workers to do inference ...
07/23/2020 14:32:33 - INFO - farm.infer -    0    0    0 
07/23/2

In [49]:
finder = Finder(reader, retriever)

In [60]:
prediction = finder.get_answers(question="Do I have to be on campus in fall?", top_k_retriever=10, top_k_reader=5)

07/23/2020 14:49:33 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.056s]
07/23/2020 14:49:33 - INFO - haystack.retriever.sparse -   Got 10 candidates from retriever
07/23/2020 14:49:33 - INFO - haystack.finder -   Reader is looking for detailed answer in 270944 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:04<00:00,  4.01s/ Batches]


In [61]:
print('Question: %s' % prediction['question'])
for answer in prediction['answers']:
    print('Answer: %s' % answer['answer'])
    print('Source: %s' % answer['meta']['name'].split('.txt')[0])
    print('Context:' '%s' % answer['context'])
    print('Score: {:0.2f}'.format(answer['probability']))
    print('------------------------------------------')

Question: Do I have to be on campus in fall?
Answer: Spring off-terms are an option
Source: students-frequently-asked-questions
Context:ummer term be available to all students who wish to attend?
Spring off-terms are an option, however there are considerations that need to be taken int
Score: 0.74
------------------------------------------
Answer: Dartmouth will be open this fall
Source: community-conversations-april-29-2020-transcript
Context:ain what I said partway through my remarks this afternoon, Dartmouth will be open this fall. We are not contemplating cancellation of fall term. Fall 
Score: 0.68
------------------------------------------
Answer: fall term is a full residential term with everyone back on campus life as usual, business as usual
Source: community-conversations-april-29-2020-transcript
Context: term and summer term, or fall term is a full residential term with everyone back on campus life as usual, business as usual.
As I sit here over these
Score: 0.66
-----------

In [56]:
print_answers(prediction, details="minimal")
prediction

[   {   'answer': 'We will not report test scores for the matriculating class '
                  'of 2025',
        'context': 'it will be better to submit a test score. We will not '
                   'report test scores for the matriculating class of 2025."\n'
                   'In addition to Mills, Helble was joined'},
    {   'answer': 'Undergraduate final exams will proceed this week',
        'context': 'ov/coronavirus/2019-ncov/php/risk-assessment.html.\n'
                   'Undergraduate final exams will proceed this week, and we '
                   'are working with faculty to provide altern'},
    {   'answer': "We'll be watching all of that closely",
        'context': 'to what we would have called a normal type of operation. '
                   "We'll be watching all of that closely.\n"
                   'Lisa, maybe you can comment a little bit more on the t'},
    {   'answer': 'Tuck will hold exams for core classes online',
        'context': 'ting to learn if th

{'question': 'Will fall exams be on campus?',
 'no_ans_gap': 3.6854190826416016,
 'answers': [{'answer': 'We will not report test scores for the matriculating class of 2025',
   'score': 8.061635971069336,
   'probability': 0.7325706752596376,
   'context': 'it will be better to submit a test score. We will not report test scores for the matriculating class of 2025."\nIn addition to Mills, Helble was joined',
   'offset_start': 42,
   'offset_end': 108,
   'offset_start_in_doc': 2884,
   'offset_end_in_doc': 2950,
   'document_id': '4ybyfHMB6KEIJEnpXvEC',
   'meta': {'name': 'community-conversations-salaries-extended-through-july-31.txt'}},
  {'answer': 'Undergraduate final exams will proceed this week',
   'score': 5.3352556228637695,
   'probability': 0.660810228610557,
   'context': 'ov/coronavirus/2019-ncov/php/risk-assessment.html.\nUndergraduate final exams will proceed this week, and we are working with faculty to provide altern',
   'offset_start': 51,
   'offset_end': 99,
   '